# Cardinality Constrained Portfolio Optimization

This notebook explores the **Cardinality-Constrained Mean-Variance (CCMV)** portfolio optimization problem, based on the methodology presented in *"Cardinality-Constrained Portfolio Optimization with Clustering"* (Ebrahimi et al., 2025).

The classical **Markowitz mean-variance model** is extended by adding a **cardinality constraint**, limiting the number of assets included in the portfolio. This transforms the problem into a **Mixed-Integer Quadratic Program (MIQP)**, making it significantly more challenging to solve due to its NP-hard nature.

In this notebook, we will:

- Formulate the classical and cardinality-constrained optimization problems
- Discuss the implications of binary variables and computational complexity
- Implement and solve small CCMV problems using Python
- Prepare for future integration of clustering methods as described in the paper

This work serves as both a theoretical deep dive and a practical foundation for reproducing the paper’s results.

### Portfolio Optimization Problem Formulation

The classical **mean-variance portfolio optimization** problem, introduced by Markowitz (1952), aims to minimize portfolio risk for a given level of expected return (or equivalently, balance both using a risk aversion parameter). It is formulated as:

$$
\min_{w \in \mathbb{R}^N} \quad w^\top \Sigma w - \gamma \mu^\top w \quad \text{subject to} \quad \mathbf{1}^\top w = 1,\quad 0 \leq w_i \leq 1 \quad \forall i \in \{1, \ldots, N\}
$$

where:

- $ w = (w_1, \ldots, w_N)^\top $ is the vector of portfolio weights  
- $ \Sigma \in \mathbb{R}^{N \times N} $ is the **covariance matrix** of asset returns  
- $ \mu \in \mathbb{R}^N $ is the **expected return vector**  
- $ \gamma > 0 $ is the **risk aversion coefficient**  
- $ \mathbf{1} \in \mathbb{R}^N $ is a vector of ones  
- The constraint $ \mathbf{1}^\top w = 1 $ ensures the portfolio is fully invested  
- The box constraint $ 0 \leq w_i \leq 1 $ limits individual asset weights

This is a **convex quadratic program** and can be efficiently solved using standard optimization methods.



This classical model is easy to solve due to its convexity, but it does not capture important practical constraints that investors often face. In real-world applications, it is common to:

- Reduce transaction costs  
- Control portfolio complexity  
- Avoid excessive concentration in a few assets

To reflect these needs, we introduce two key extensions:

#### 1. Cardinality Constraint

Investors may want to limit the total number of assets in the portfolio. Let $\Delta \in \mathbb{N}_+$ denote the maximum allowed number of assets. This leads to the **cardinality constraint**:

$$
\|w\|_0 \leq \Delta
$$

Here, $\|w\|_0$ is the **zero-norm**, which counts the number of nonzero entries in $w$. This constraint is **non-convex** and introduces **combinatorial complexity**, requiring the use of binary decision variables.

#### 2. Box Constraint on Asset Weights

To prevent overexposure to individual assets, we constrain each weight to lie within a maximum threshold. Let $\bar{w} \in (0, 1]$ be the maximum allowable weight per asset. The constraint is written as:

$$
w \in [0, \bar{w}]^N
$$

This **box constraint** ensures no single asset can dominate the portfolio, and also helps with regulatory compliance or risk diversification.

Together, these constraints transform the problem into a **Mixed-Integer Quadratic Program (MIQP)**, which is computationally much harder to solve than the unconstrained case.

We can then formulate the **Cardinality-Constrained Mean-Variance (CCMV)** optimization problem as follows:

$$
w^*(\Delta, \bar{w}) = \arg\min_{w \in \mathbb{R}^N} \quad w^\top \Sigma w - \gamma \mu^\top w \quad \text{subject to} \quad \mathbf{1}^\top w = 1, \quad 0 \leq w_i \leq \bar{w} \quad \forall i, \quad \|w\|_0 \leq \Delta
$$

Furthermore, it can be shown that this problem is NP-hard due to the combinatorial nature of the cardinality constraint. The introduction of binary variables to enforce this constraint complicates the optimization significantly.
